<a href="https://colab.research.google.com/github/Vecheslav-Eremeev/keras_projects/blob/main/Titanic_test_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from catboost import CatBoostClassifier, Pool, cv

Load Data

In [5]:
data = sns.load_dataset('titanic')
data.shape

(891, 15)

In [6]:
data.keys()

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [7]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [8]:
data.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

Data preparation

In [9]:
data['family'] = data['sibsp'] + data['parch'] + 1

In [12]:
drop_col = 'survived class who adult_male deck embark_town alive alone'.split()

In [13]:
X = data.drop(drop_col, axis=1)
y = data['survived']

In [16]:
X['age'] = X['age'].fillna(X['age'].mean())

In [17]:
X['embarked'] = X['embarked'].fillna(X['embarked'].mode()[0])

In [18]:
X.isnull().sum()

pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
family      0
dtype: int64

In [21]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]
categorical_features_indices

array([0, 1, 3, 4, 6, 7])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

Train Model

In [23]:
model = CatBoostClassifier(custom_loss=['Accuracy'], random_seed=42, logging_level='Silent')

In [24]:
model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_test, y_test), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
pred = model.predict(X_test)

In [27]:
print(accuracy_score(y_test, pred))

0.8161434977578476
